In [1]:
import numpy as np
from collections import Counter

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_provider import IBMProvider
from qiskit.primitives import Sampler

from qiskit.visualization import plot_histogram
from qiskit.opflow import (
    CircuitStateFn,
    StateFn,
    PauliExpectation,
    CircuitSampler
)


In [ ]:
## Aaronson-Inspired Certified Randomness

#This notebook explores randomness generation using structured non-Clifford quantum circuits, inspired by Aaronson-style certified randomness and cross-entropy benchmarking.

#Rather than assuming ideal randomness, the approach analyses observable statistics from real quantum hardware, comparing ideal, noisy, and experimental behaviour under explicit assumptions.


In [ ]:
# Circuit parameters
num_qubits = 60
depth = 7
alpha = 0.0005  # distribution shaping parameter


In [ ]:
def excitation_preserving_gate(circ, q1, q2, theta, phi):
    circ.rz(theta, q1)
    circ.rz(theta, q2)
    circ.rxx(theta, q1, q2)
    circ.ryy(theta, q1, q2)
    circ.cp(phi, q1, q2)


In [ ]:
def non_clifford_random_circuit(num_qubits, depth, alpha):
    circ = QuantumCircuit(num_qubits)
    
    for q in range(0, num_qubits, 2):
        circ.x(q)

    for layer in range(depth):
        for q in range(num_qubits):
            theta = np.pi * (2.53 * q if q % 2 == 0 else -2.47 * q)
            circ.rz(theta, q)

        for q in range(layer % 2, num_qubits - 1, 2):
            excitation_preserving_gate(
                circ,
                q,
                q + 1,
                theta=np.pi / (2 * (1 + alpha / (q * layer + 1))),
                phi=np.pi / 6
            )

    circ.h(range(num_qubits))
    circ.measure_all()
    return circ


In [ ]:
qc = non_clifford_random_circuit(num_qubits, depth, alpha)
qc


In [ ]:
provider = IBMProvider()  # assumes credentials are set via environment
real_backend = provider.get_backend("ibm_brisbane")

noise_model = NoiseModel.from_backend(real_backend)
noisy_simulator = AerSimulator(noise_model=noise_model)


In [ ]:
# Ideal simulation
ideal_sampler = Sampler()
ideal_job = ideal_sampler.run(qc)
ideal_dist = ideal_job.result().quasi_dists[0]

# Noisy simulation
noisy_qc = transpile(qc, noisy_simulator)
noisy_result = noisy_simulator.run(noisy_qc).result()
noisy_counts = noisy_result.get_counts()

# Real hardware
real_qc = transpile(qc, real_backend)
real_job = real_backend.run(real_qc, shots=20000)
real_counts = real_job.result().get_counts()


In [ ]:
plot_histogram(
    [ideal_dist, noisy_counts, real_counts],
    legend=["Ideal", "Noisy simulator", "Real hardware"]
)


In [ ]:
def cross_entropy_benchmark(counts, num_qubits, depth, alpha):
    backend = AerSimulator()
    total = 0.0

    for bitstring in counts:
        init = QuantumCircuit(num_qubits)
        for i, bit in enumerate(bitstring):
            if bit == "1":
                init.x(i)

        random_op = non_clifford_random_circuit(num_qubits, depth, alpha)
        measurable = StateFn(random_op, is_measurement=True).compose(
            CircuitStateFn(init)
        )

        expectation = PauliExpectation().convert(measurable)
        sampler = CircuitSampler(backend).convert(expectation)
        val = sampler.eval().real
        total += val ** 2

    return total


In [ ]:
## Discussion and Limitations

#The cross-entropy score provides a quantitative indicator of how closely experimental outcomes track ideal quantum behaviour. While this does not constitute device-independent certification, it aligns with Aaronson-style reasoning: randomness is inferred from statistical structure rather than assumed.

#The value of this work lies in methodology- combining circuit complexity, experimental execution, and statistical analysis under realistic noise.
